In [1]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
def order_book(month,day):
    data1 = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/stat_order_book/CNF14_0'+str(month)+'_'+str(day)+'_order_book_final.csv'
    data1 = pd.read_csv(datapath,sep='\t',encoding = 'utf-8')
    data_book = data1[['0','1','2','3']]
    return data_book

In [3]:
def day_time(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.0'+str(month)+'.'+str(day)+'.csv'
    data = pd.read_csv(datapath)
    data_CNF14 = data[data.Series == 'CNF14']
    data = data_CNF14
    market_open_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D09:00')].index.tolist()[0] 
    market_close_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D16:00')].index.tolist()[0] 
    data_open = data[market_open_time:market_close_time + 1]
    timestamp_ = data_open.TimeStamp.unique()
    return timestamp_

In [4]:
def time_transform(timestamp_time):
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600 \
                +float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60\
                +float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second.append(second - 32400.0)
    return  time_second

In [5]:
def bid123_ask123_Q(data_book_28_open):
    Bid1 = []
    Bid2 = []
    Bid3 = []
    Bid1_Quantity = []
    Bid2_Quantity = []
    Bid3_Quantity = []
    Ask1 = []
    Ask2 = []
    Ask3 = []
    Ask1_Quantity = []
    Ask2_Quantity = []
    Ask3_Quantity = []

    TimeStamp = []
    for i in range(1,len(data_book_28_open),4):
        #print data_book_28_open.iloc[i]['0']
        #print data_book_28_open.iloc[i]['2']
        Bid1.append(float(data_book_28_open.iloc[i]['0'])/100.0)
        Bid1_Quantity.append(float(data_book_28_open.iloc[i]['1']))

        Bid2.append(float(data_book_28_open.iloc[i + 1]['0'])/100.0)
        Bid2_Quantity.append(float(data_book_28_open.iloc[i + 1]['1']))  

        Bid3.append(float(data_book_28_open.iloc[i + 2]['0'])/100.0)
        Bid3_Quantity.append(float(data_book_28_open.iloc[i + 2]['1']))  

        Ask1.append(float(data_book_28_open.iloc[i]['2'])/100.0) 
        Ask1_Quantity.append(float(data_book_28_open.iloc[i]['3']))

        Ask2.append(float(data_book_28_open.iloc[i + 1]['2'])/100.0) 
        Ask2_Quantity.append(float(data_book_28_open.iloc[i + 1]['3']))    

        Ask3.append(float(data_book_28_open.iloc[i + 2]['2'])/100.0) 
        Ask3_Quantity.append(float(data_book_28_open.iloc[i + 2]['3']))    
        TimeStamp.append(data_book_28_open.iloc[i-1][1])
    return Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity,    

In [6]:
def rise_ask(Ask1,timestamp_time_second):
    rise_ratio = []
    index = np.where(np.array(timestamp_time_second) >= 600)[0][0]
    for i in range(0,index):
        rise_ratio_ = round((Ask1[i] - Ask1[0])*(1.0)/Ask1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Ask1),1):
        #print timestamp_time_second[:i]
        #print timestamp_time_second[i] - 600
        #print np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        index_start = np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        rise_ratio_ = round((Ask1[i] - Ask1[index_start])*(1.0)/Ask1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return rise_ratio


In [7]:
def rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,close_price,\
             N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,\
             N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
             N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI):
    
    if rise_ratio_ask[index] <= -0.4:
        N_1 = N_1 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_1_T = N_1_T + 1
            N_1_E = N_1_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_1_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_1_TI = N_1_TI + (time_traded - time_start)
            #print N_1_TI
            pass

        else:
            if i < 25200 - 900:
                N_1_E = N_1_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                #print N_1_M
                N_1_TI = N_1_TI + (900.0)
                #print N_1_TI
            else:
                N_1_E = N_1_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0) 
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_1_TI = N_1_TI + (time_traded - time_start)
                #print N_1_TI
            pass

    elif -0.4 < rise_ratio_ask[index] <= -0.15:
        N_2 = N_2 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 3
            N_2_T = N_2_T + 1
            N_2_E = N_2_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_2_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_2_TI = N_2_TI + (time_traded - time_start)
            #print N_2_TI
            pass
        else:
            if i < 25200 - 900:
                N_2_E = N_2_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_2_TI = N_2_TI + (900.0)
                #print N_2_TI
            else:
                N_2_E = N_2_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_2_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_2_TI = N_2_TI + (time_traded - time_start)
                #print N_2_TI
            pass

    elif -0.15 < rise_ratio_ask[index] <= 0.15:
        N_3 = N_3 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 5
            N_3_T = N_3_T + 1
            N_3_E = N_3_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_3_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_3_TI = N_3_TI + (time_traded - time_start)
            #print N_3_TI
            pass
        else:
            if i < 25200 - 900:
                N_3_E = N_3_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_3_TI = N_3_TI + (900.0)
                #print N_3_TI
            else:
                N_3_E = N_3_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_3_TI = N_3_TI + (time_traded - time_start)
                #print N_3_TI
            pass

    elif 0.15 < rise_ratio_ask[index] < 0.4:
        N_4 = N_4 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 7
            N_4_T = N_4_T + 1
            N_4_E = N_4_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_4_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_4_TI = N_4_TI + (time_traded - time_start)
            #print N_4_TI
            pass
        else:
            if i < 25200 - 900:
                N_4_E = N_4_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_4_TI = N_4_TI + (900.0)
                #print N_4_TI
            else:
                N_4_E = N_4_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_4_TI = N_4_TI + (time_traded - time_start)
                #print N_4_TI
            pass

    elif 0.4 <= rise_ratio_ask[index]:
        N_5 = N_5 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_5_T = N_5_T + 1
            N_5_E = N_5_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_5_E      
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_5_TI = N_5_TI + (time_traded - time_start)
            #print N_5_TI
            pass
        else:
            if i < 25200 - 900:
                N_5_E = N_5_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_1_TI = N_1_TI + (900.0)
                #print N_5_TI
            else:
                N_5_E = N_5_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_5_TI = N_5_TI + (time_traded - time_start)
                #print N_5_TI 
            pass
        
    return N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
           N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI


In [16]:
def bid(time1,time2,Bid,Ask,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,\
        Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weigth1,weigth2,weigth3):
    [N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
     N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI] = [0 for i in  range(0,25,1)]
    [N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
     N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI] = [0 for i in  range(0,25,1)]
    [N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
     N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI] = [0 for i in  range(0,25,1)]
    [N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
     N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI] = [0 for i in  range(0,25,1)]
    [N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
     N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI] = [0 for i in  range(0,25,1)]

    for i in range(time1,time2,1):
        #print i
        #print np.where(array(timestamp_time_second) <= i)[0]
        index = np.where(array(timestamp_time_second) <= i)[0][-1]
        #print i
        #print index
        #print timestamp_time_second[index]
        if i < 25200 - 900:
            index_15_min = np.where(np.array(timestamp_time_second[index:]) >= i + 900)[0][0]
            #print index_15_min
            #print timestamp_time_second[index+index_15_min]
        #print 'time stamp = %d'%(timestamp_time_second[index])
        #print index
        #print index + index_15_min 
        #rint Bid1[index]
        #print Ask1[index]
        #print len(Ask1[(index+1):index + index_15_min])

        #if Bid1[index] > min(Ask1[(index+1):index + index_15_min]):
        #    first_min_index = np.where(np.array(Ask1[(index+1):index + index_15_min]) < Bid1[index])[0][0]
        #    print 'mdd = %d'%((max(Ask1[(index + 1):(index + 1 + first_min_index)]) - Bid1[index])*(1.0)/(5.0))
        #print min_Ask1_time_series[index + 1]
        weighted_Bid = Bid1_Quantity[index] * weight1 + Bid2_Quantity[index] * weight2 + Bid3_Quantity[index] * weight3
        weighted_Ask = Ask1_Quantity[index] * weight1 + Ask2_Quantity[index] * weight2 + Ask3_Quantity[index] * weight3  
        #print weighted_Bid 
        #print weighted_Ask
        #print (weighted_Bid*(1.0) / weighted_Ask)

        if (weighted_Bid*(1.0) / weighted_Ask) <= 0.7:
            N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
            N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,\
                                                          N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,\
                                                          N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI)                                               

        elif 0.7 < (weighted_Bid*(1.0) / weighted_Ask) <= 0.9:
            N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
            N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,\
                                                          N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,\
                                                          N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI)                                                

        elif 0.9 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.1:
            N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
            N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,\
                                                          N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,\
                                                          N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI)                                    

        elif 1.1 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.3:
            N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
            N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,\
                                                          N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,\
                                                          N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI)                              

        elif 1.30 < (weighted_Bid*(1.0) / weighted_Ask):
            N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
            N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,\
                                                          N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,\
                                                          N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI)                 

    return [[N_11,N_12,N_13,N_14,N_15],[N_11_T,N_12_T,N_13_T,N_14_T,N_15_T],[N_11_E,N_12_E,N_13_E,N_14_E,N_15_E],\
             [N_11_M,N_12_M,N_13_M,N_14_M,N_15_M],[N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI],\
            [N_21,N_22,N_23,N_24,N_25],[N_21_T,N_22_T,N_23_T,N_24_T,N_25_T],[N_21_E,N_22_E,N_23_E,N_24_E,N_25_E],\
             [N_21_M,N_22_M,N_23_M,N_24_M,N_25_M],[N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI],\
            [N_31,N_32,N_33,N_34,N_35],[N_31_T,N_32_T,N_33_T,N_34_T,N_35_T],[N_31_E,N_32_E,N_33_E,N_34_E,N_35_E],\
             [N_31_M,N_32_M,N_33_M,N_34_M,N_35_M],[N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI],\
            [N_41,N_42,N_43,N_44,N_45],[N_41_T,N_42_T,N_43_T,N_44_T,N_45_T],[N_41_E,N_42_E,N_43_E,N_44_E,N_45_E],\
             [N_41_M,N_42_M,N_43_M,N_44_M,N_45_M],[N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI],\
            [N_51,N_52,N_53,N_54,N_55],[N_51_T,N_52_T,N_53_T,N_54_T,N_55_T],[N_51_E,N_52_E,N_53_E,N_54_E,N_55_E],\
             [N_51_M,N_52_M,N_53_M,N_54_M,N_55_M],[N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI]] 


### day 1/28

In [42]:
data_book = order_book(1,28)
data_book_open = data_book[213*4-3:67229*4+1]
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,28)
timestamp_time_second = time_transform(timestamp_time)

In [43]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 100.754322


In [44]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 70.863560


9:00~9:15

In [45]:
close_price = 6750
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [46]:
time1 = 0
time2 = 600
mat28_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [47]:
mat28_1_15,mat28_2_15, mat28_3_15

([[0, 0, 0, 11, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, -43.0, 0],
  [0, 0, 0, 61.0, 0],
  [0, 0, 0, 9405.0, 0],
  [0, 0, 3, 41, 0],
  [0, 0, 0, 39, 0],
  [0, 0, -15.0, 31.0, 0],
  [0, 0, 21.0, 53.0, 0],
  [0, 0, 2700.0, 22819.0, 0],
  [0, 0, 0, 57, 0],
  [0, 0, 0, 11, 0],
  [0, 0, 0, -246.0, 0],
  [0, 0, 0, 328.0, 0],
  [0, 0, 0, 47553.0, 0],
  [0, 0, 0, 164, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, -789.0, 0],
  [0, 0, 0, 1093.0, 0],
  [0, 0, 0, 147600.0, 0],
  [0, 0, 113, 211, 0],
  [0, 0, 0, 19, 0],
  [0, 0, -660.0, -830.0, 0],
  [0, 0, 882.0, 1326.0, 0],
  [0, 0, 101700.0, 180728.0, 0]],
 [[0, 0, 0, 11, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, -48.0, 0],
  [0, 0, 0, 77.0, 0],
  [0, 0, 0, 9900.0, 0],
  [0, 0, 3, 41, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -15.0, -170.0, 0],
  [0, 0, 24.0, 279.0, 0],
  [0, 0, 2700.0, 36900.0, 0],
  [0, 0, 0, 57, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, -301.0, 0],
  [0, 0, 0, 429.0, 0],
  [0, 0, 0, 51300.0, 0],
  [0, 0, 0, 164, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, -789.0, 0],
  [0,

9:15~11:00

In [48]:
time1 = 600
time2 = 7200
mat28_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

11:00~13:00

In [49]:
time1 = 7200
time2 = 14400
mat28_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

13:00~16:00

In [50]:
time1 = 14400
time2 = 25200
mat28_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/29

In [51]:
data_book = order_book(1,29)
data_book_open = data_book[602:128801+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,29)
timestamp_time_second = time_transform(timestamp_time)

In [52]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 44.927615


In [53]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 31.245102


In [54]:
close_price = 6775
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [55]:
time1 = 0
time2 = 600
mat29_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [56]:
mat29_1_15,mat29_2_15,mat29_3_15

([[0, 0, 406, 0, 0],
  [0, 0, 12, 0, 0],
  [0, 0, -1101.0, 0, 0],
  [0, 0, 2112.0, 0, 0],
  [0, 0, 357841.0, 0, 0],
  [0, 0, 37, 0, 0],
  [0, 0, 13, 0, 0],
  [0, 0, -88.0, 0, 0],
  [0, 0, 193.0, 0, 0],
  [0, 0, 25689.0, 0, 0],
  [0, 0, 9, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -32.0, 0, 0],
  [0, 0, 54.0, 0, 0],
  [0, 0, 8100.0, 0, 0],
  [0, 0, 105, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -268.0, 0, 0],
  [0, 0, 613.0, 0, 0],
  [0, 0, 94500.0, 0, 0],
  [0, 0, 43, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -55.0, 0, 0],
  [0, 0, 255.0, 0, 0],
  [0, 0, 38700.0, 0, 0]],
 [[0, 0, 406, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -1117.0, 0, 0],
  [0, 0, 2554.0, 0, 0],
  [0, 0, 365400.0, 0, 0],
  [0, 0, 37, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -128.0, 0, 0],
  [0, 0, 230.0, 0, 0],
  [0, 0, 33300.0, 0, 0],
  [0, 0, 9, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -32.0, 0, 0],
  [0, 0, 63.0, 0, 0],
  [0, 0, 8100.0, 0, 0],
  [0, 0, 105, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, -268.0, 0, 0],
  [0, 0, 718.0, 0, 0],
  [0, 0, 94500.0, 0, 0]

In [57]:
time1 = 600
time2 = 7200
mat29_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [58]:
time1 = 7200
time2 = 14400
mat29_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [59]:
time1 = 14400
time2 = 25200
mat29_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/27

In [60]:
data_book = order_book(1,27)
data_book_open = data_book[1112:266575+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,27)
timestamp_time_second = time_transform(timestamp_time)

In [61]:
len(timestamp_time_second)

66366

In [62]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 90.740464


In [63]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 154.496044


In [64]:
close_price = 6710
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [65]:
time1 = 0
time2 = 600
mat27_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [66]:
time1 = 600
time2 = 7200
mat27_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [67]:
time1 = 7200
time2 = 14400
mat27_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [68]:
time1 = 14400
time2 = 25200
mat27_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [39]:
#pd.DataFrame(array(mat29_1)+array(mat28_1)+array(mat27_1))

In [69]:
a = 0
array(mat[a+1])*1.0/array(mat[a]),array(mat[a+2])/array(mat[a]),array(mat[a+3])/array(mat[a]),array(mat[a+4])/array(mat[a])

(array([ 0.5297619 ,  0.31147541,  0.54868819,  0.88974359,  1.        ]),
 array([-0.88095238, -0.22404372,  0.40665994,  0.96282051,  1.        ]),
 array([ 3.07738095,  3.01639344,  2.01437941,  2.08717949,  1.        ]),
 array([ 450.7202381 ,  657.93989071,  555.90792129,  428.22051282,
         163.85714286]))

### Day 1/24

In [69]:
data_book = order_book(1,24)
data_book_open = data_book[920:331143+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,24)
timestamp_time_second = time_transform(timestamp_time)

In [81]:
#data_book_open

In [75]:
len(timestamp_time_second)

82556

In [71]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 165.940777


In [83]:
len(Ask1)

82556

In [72]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 386.123375


In [73]:
close_price = 6815
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [74]:
time1 = 0
time2 = 600
mat24_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [75]:
time1 = 600
time2 = 7200
mat24_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [76]:
time1 = 7200
time2 = 14400
mat24_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [77]:
time1 = 14400
time2 = 25200
mat24_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/22

In [78]:
data_book = order_book(1,22)
data_book_open = data_book[1304:368779+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,22)
timestamp_time_second = time_transform(timestamp_time)

In [79]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 125.253136


In [80]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 393.654061


In [81]:
close_price = 6870
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [82]:
time1 = 0
time2 = 600
mat22_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [83]:
time1 = 600
time2 = 7200
mat22_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [84]:
time1 = 7200
time2 = 14400
mat22_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [85]:
time1 = 14400
time2 = 25200
mat22_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/21

In [86]:
data_book = order_book(1,21)
data_book_open = data_book[1236:351679+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,21)
timestamp_time_second = time_transform(timestamp_time)

In [87]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 121.217293


In [88]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 409.557436


In [89]:
close_price = 6710
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [90]:
time1 = 0
time2 = 600
mat21_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [91]:
time1 = 600
time2 = 7200
mat21_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [92]:
time1 = 7200
time2 = 14400
mat21_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [93]:
time1 = 14400
time2 = 25200
mat21_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/20

In [94]:
data_book = order_book(1,20)
data_book_open = data_book[1300:332255+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,20)
timestamp_time_second = time_transform(timestamp_time)

In [95]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 112.028675


In [96]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 237.243867


In [97]:
close_price = 6655
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [98]:
time1 = 0
time2 = 600
mat20_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [99]:
time1 = 600
time2 = 7200
mat20_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [100]:
time1 = 7200
time2 = 14400
mat20_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [101]:
time1 = 14400
time2 = 25200
mat20_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/17

In [170]:
data_book = order_book(1,17)
data_book_open = data_book[968:377687+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,17)
timestamp_time_second = time_transform(timestamp_time)

In [107]:
#data_book_open

In [171]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 183.318054


In [172]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 542.924225


In [173]:
close_price = 6680
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [174]:
time1 = 0
time2 = 600
mat17_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [112]:
time1 = 600
time2 = 7200
mat17_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [113]:
time1 = 7200
time2 = 14400
mat17_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [114]:
time1 = 14400
time2 = 25200
mat17_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/16

In [115]:
data_book = order_book(1,16)
data_book_open = data_book[1380:285495+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,16)
timestamp_time_second = time_transform(timestamp_time)

In [116]:
#data_book_open

,0,1,2,3
0,TimeStamp,09:00:00.045499800,NaN,NaN
1,677000,5,677500.0,5.0
2,676500,28,678000.0,76.0
3,676000,28,678500.0,68.0
4,TimeStamp,09:00:00.046499800,NaN,NaN
5,677000,5,677500.0,5.0
6,676500,28,678000.0,76.0
7,676000,38,678500.0,68.0
8,TimeStamp,09:00:00.099498400,NaN,NaN
9,677000,5,677500.0,5.0


In [117]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 101.965176


In [118]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 261.835881


In [119]:
close_price = 6750
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [120]:
time1 = 0
time2 = 600
mat16_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [121]:
time1 = 600
time2 = 7200
mat16_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [122]:
time1 = 7200
time2 = 14400
mat16_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [123]:
time1 = 14400
time2 = 25200
mat16_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [124]:
SUM_Bid1_15 = \
array(mat16_1_15)+array(mat17_1_15)+array(mat20_1_15)+array(mat21_1_15)+array(mat22_1_15)+\
array(mat24_1_15)+array(mat27_1_15)+array(mat28_1_15)+array(mat29_1_15)
SUM_Bid2_15 = \
array(mat16_2_15)+array(mat17_2_15)+array(mat20_2_15)+array(mat21_2_15)+array(mat22_2_15)+\
array(mat24_2_15)+array(mat27_2_15)+array(mat28_2_15)+array(mat29_2_15)
SUM_Bid3_15 = \
array(mat16_3_15)+array(mat17_3_15)+array(mat20_3_15)+array(mat21_3_15)+array(mat22_3_15)+\
array(mat24_3_15)+array(mat27_3_15)+array(mat28_3_15)+array(mat29_3_15)

In [175]:
SUM_Bid1_11 = \
array(mat16_1_11)+array(mat17_1_11)+array(mat20_1_11)+array(mat21_1_11)+array(mat22_1_11)+\
array(mat24_1_11)+array(mat27_1_11)+array(mat28_1_11)+array(mat29_1_11)
SUM_Bid2_11 = \
array(mat16_2_11)+array(mat17_2_11)+array(mat20_2_11)+array(mat21_2_11)+array(mat22_2_11)+\
array(mat24_2_11)+array(mat27_2_11)+array(mat28_2_11)+array(mat29_2_11)
SUM_Bid3_11 = \
array(mat16_3_11)+array(mat17_3_11)+array(mat20_3_11)+array(mat21_3_11)+array(mat22_3_11)+\
array(mat24_3_11)+array(mat27_3_11)+array(mat28_3_11)+array(mat29_3_11)

In [127]:
SUM_Bid1_13 = \
array(mat16_1_13)+array(mat17_1_13)+array(mat20_1_13)+array(mat21_1_13)+array(mat22_1_13)+\
array(mat24_1_13)+array(mat27_1_13)+array(mat28_1_13)+array(mat29_1_13)
SUM_Bid2_13 = \
array(mat16_2_13)+array(mat17_2_13)+array(mat20_2_13)+array(mat21_2_13)+array(mat22_2_13)+\
array(mat24_2_13)+array(mat27_2_13)+array(mat28_2_13)+array(mat29_2_13)
SUM_Bid3_13 = \
array(mat16_3_13)+array(mat17_3_13)+array(mat20_3_13)+array(mat21_3_13)+array(mat22_3_13)+\
array(mat24_3_13)+array(mat27_3_13)+array(mat28_3_13)+array(mat29_3_13)

In [141]:
SUM_Bid1_16 = \
array(mat16_1_16)+array(mat17_1_16)+array(mat20_1_16)+array(mat21_1_16)+array(mat22_1_16)+\
array(mat24_1_16)+array(mat27_1_16)+array(mat28_1_16)+array(mat29_1_16)
SUM_Bid2_16 = \
array(mat16_2_16)+array(mat17_2_16)+array(mat20_2_16)+array(mat21_2_16)+array(mat22_2_16)+\
array(mat24_2_16)+array(mat27_2_16)+array(mat28_2_16)+array(mat29_2_16)
SUM_Bid3_16 = \
array(mat16_3_16)+array(mat17_3_16)+array(mat20_3_16)+array(mat21_3_16)+array(mat22_3_16)+\
array(mat24_3_16)+array(mat27_3_16)+array(mat28_3_16)+array(mat29_3_16)

In [129]:
SUM_Bid1_15

array([[  1.20000000e+02,   2.64000000e+02,   1.67800000e+03,
          3.09000000e+02,   2.10000000e+01],
       [  2.10000000e+01,   2.47000000e+02,   7.97000000e+02,
          2.81000000e+02,   2.10000000e+01],
       [ -1.90000000e+02,   1.68000000e+02,  -5.39000000e+02,
          1.93000000e+02,   2.30000000e+01],
       [  5.27000000e+02,   4.18000000e+02,   5.57600000e+03,
          8.17000000e+02,   2.80000000e+01],
       [  9.20430000e+04,   7.37300000e+04,   1.03451900e+06,
          1.09316000e+05,   1.40300000e+03],
       [  1.57000000e+02,   3.24000000e+02,   1.04700000e+03,
          2.28000000e+02,   1.90000000e+01],
       [  4.80000000e+01,   2.75000000e+02,   8.01000000e+02,
          1.81000000e+02,   1.90000000e+01],
       [ -2.60000000e+02,   7.50000000e+01,   4.01000000e+02,
          9.30000000e+01,   2.00000000e+01],
       [  6.84000000e+02,   7.99000000e+02,   3.08100000e+03,
          6.30000000e+02,   2.80000000e+01],
       [  1.04804000e+05,   9.2840000

In [130]:
SUM_Bid2_15

array([[  1.20000000e+02,   2.64000000e+02,   1.67800000e+03,
          3.09000000e+02,   2.10000000e+01],
       [  5.00000000e+00,   2.31000000e+02,   6.96000000e+02,
          1.31000000e+02,   1.60000000e+01],
       [ -2.45000000e+02,   3.75000000e+02,  -2.30000000e+01,
         -4.00000000e+01,   4.10000000e+01],
       [  7.03000000e+02,   7.07000000e+02,   7.60400000e+03,
          1.31900000e+03,   5.40000000e+01],
       [  1.10493000e+05,   1.01509000e+05,   1.15196500e+06,
          2.21399000e+05,   1.67800000e+03],
       [  1.57000000e+02,   3.24000000e+02,   1.04700000e+03,
          2.28000000e+02,   1.90000000e+01],
       [  3.00000000e+01,   2.31000000e+02,   6.86000000e+02,
          1.19000000e+02,   1.00000000e+01],
       [ -2.86000000e+02,   2.23000000e+02,   7.98000000e+02,
         -1.40000000e+01,   3.00000000e+01],
       [  8.97000000e+02,   1.20700000e+03,   4.54500000e+03,
          1.12100000e+03,   5.40000000e+01],
       [  1.31471000e+05,   1.3827000

In [131]:
SUM_Bid3_15

array([[  1.20000000e+02,   2.64000000e+02,   1.67800000e+03,
          3.09000000e+02,   2.10000000e+01],
       [  1.00000000e+00,   8.60000000e+01,   6.49000000e+02,
          8.40000000e+01,   3.00000000e+00],
       [ -2.52000000e+02,   1.74000000e+02,   4.79000000e+02,
          3.40000000e+01,   4.20000000e+01],
       [  8.23000000e+02,   1.36300000e+03,   9.42800000e+03,
          1.64300000e+03,   7.50000000e+01],
       [  1.23315000e+05,   1.78283000e+05,   1.21512700e+06,
          2.48226000e+05,   9.88000000e+02],
       [  1.57000000e+02,   3.24000000e+02,   1.04700000e+03,
          2.28000000e+02,   1.90000000e+01],
       [  1.20000000e+01,   1.69000000e+02,   5.38000000e+02,
          9.40000000e+01,   8.00000000e+00],
       [ -3.14000000e+02,   2.54000000e+02,   9.55000000e+02,
          8.20000000e+01,   3.80000000e+01],
       [  1.06000000e+03,   1.64300000e+03,   5.85400000e+03,
          1.34900000e+03,   7.30000000e+01],
       [  1.43182000e+05,   1.7865600

In [176]:
SUM_Bid1_11

array([[  1.95000000e+02,   6.74000000e+02,   5.40600000e+03,
          2.26200000e+03,   5.34000000e+02],
       [  1.10000000e+02,   4.61000000e+02,   4.29800000e+03,
          1.65400000e+03,   4.68000000e+02],
       [ -1.53000000e+02,  -3.30000000e+01,   1.67700000e+03,
         -4.29000000e+02,   3.59000000e+02],
       [  6.02000000e+02,   1.81900000e+03,   1.44110000e+04,
          7.52800000e+03,   1.38200000e+03],
       [  1.46067000e+05,   2.88837000e+05,   2.17490000e+06,
          1.00707200e+06,   8.43070000e+04],
       [  1.84000000e+02,   1.23100000e+03,   5.76600000e+03,
          2.22100000e+03,   6.28000000e+02],
       [  9.90000000e+01,   8.23000000e+02,   3.61500000e+03,
          1.43600000e+03,   5.67000000e+02],
       [ -1.69000000e+02,  -5.70000000e+01,  -2.85400000e+03,
         -1.27200000e+03,   4.63000000e+02],
       [  6.69000000e+02,   3.58500000e+03,   1.99450000e+04,
          8.23000000e+03,   1.87300000e+03],
       [  1.58612000e+05,   5.6753100

In [177]:
SUM_Bid2_11

array([[  1.95000000e+02,   6.74000000e+02,   5.40600000e+03,
          2.26200000e+03,   5.34000000e+02],
       [  8.20000000e+01,   3.10000000e+02,   3.20900000e+03,
          1.34200000e+03,   3.81000000e+02],
       [ -1.43000000e+02,  -4.40000000e+01,   2.67800000e+03,
          1.47000000e+02,   5.54000000e+02],
       [  8.72000000e+02,   2.81500000e+03,   2.17610000e+04,
          1.06110000e+04,   2.30500000e+03],
       [  2.54639000e+05,   4.13436000e+05,   3.18980900e+06,
          1.49955000e+06,   1.19742000e+05],
       [  1.84000000e+02,   1.23100000e+03,   5.76600000e+03,
          2.22100000e+03,   6.28000000e+02],
       [  8.30000000e+01,   6.01000000e+02,   2.59200000e+03,
          8.38000000e+02,   4.59000000e+02],
       [ -1.27000000e+02,   1.45000000e+02,  -2.94900000e+03,
         -2.07100000e+03,   6.56000000e+02],
       [  8.94000000e+02,   5.14800000e+03,   2.79510000e+04,
          1.21700000e+04,   2.94400000e+03],
       [  2.79908000e+05,   7.9702900

In [178]:
SUM_Bid3_11

array([[  1.95000000e+02,   6.74000000e+02,   5.40600000e+03,
          2.26200000e+03,   5.34000000e+02],
       [  1.80000000e+01,   2.65000000e+02,   2.66600000e+03,
          5.76000000e+02,   3.52000000e+02],
       [ -3.01000000e+02,   1.53000000e+02,   4.20000000e+03,
          2.20000000e+01,   8.78000000e+02],
       [  1.23800000e+03,   3.53400000e+03,   2.79480000e+04,
          1.32440000e+04,   2.87700000e+03],
       [  3.29842000e+05,   4.80179000e+05,   3.72975700e+06,
          1.80924200e+06,   1.61939000e+05],
       [  1.84000000e+02,   1.23100000e+03,   5.76600000e+03,
          2.22100000e+03,   6.28000000e+02],
       [  5.90000000e+01,   3.52000000e+02,   1.95500000e+03,
          4.96000000e+02,   4.10000000e+02],
       [ -6.90000000e+01,   2.24000000e+02,  -2.28300000e+03,
         -2.24000000e+03,   1.01100000e+03],
       [  1.09200000e+03,   6.48600000e+03,   3.45530000e+04,
          1.49900000e+04,   3.61800000e+03],
       [  3.36000000e+05,   9.8534400

In [135]:
SUM_Bid1_13

array([[  5.80000000e+01,   2.02000000e+02,   1.51590000e+04,
          8.87000000e+02,   1.05000000e+02],
       [  0.00000000e+00,   8.20000000e+01,   4.89100000e+03,
          4.68000000e+02,   9.60000000e+01],
       [ -2.39000000e+02,  -2.47000000e+02,   5.86000000e+03,
          3.90000000e+01,   9.30000000e+01],
       [  3.30000000e+02,   6.01000000e+02,   2.53440000e+04,
          2.14100000e+03,   1.39000000e+02],
       [  6.03000000e+04,   1.26772000e+05,   1.17126420e+07,
          5.04225000e+05,   1.14620000e+04],
       [  4.90000000e+01,   3.00000000e+02,   6.68200000e+03,
          7.07000000e+02,   1.44000000e+02],
       [  0.00000000e+00,   9.20000000e+01,   1.90700000e+03,
          3.79000000e+02,   1.21000000e+02],
       [ -2.14000000e+02,  -3.68000000e+02,  -9.48000000e+02,
          2.07000000e+02,   1.10000000e+02],
       [  3.03000000e+02,   9.00000000e+02,   1.61600000e+04,
          2.30200000e+03,   2.46000000e+02],
       [  6.48000000e+04,   2.0756800

In [136]:
SUM_Bid2_13

array([[  5.80000000e+01,   2.02000000e+02,   1.51590000e+04,
          8.87000000e+02,   1.05000000e+02],
       [  0.00000000e+00,   1.50000000e+01,   1.04300000e+03,
          2.49000000e+02,   8.00000000e+01],
       [ -2.39000000e+02,  -3.62000000e+02,   6.13700000e+03,
          9.90000000e+01,   1.55000000e+02],
       [  3.88000000e+02,   8.59000000e+02,   4.11760000e+04,
          3.14900000e+03,   2.50000000e+02],
       [  7.47000000e+04,   1.69259000e+05,   1.31939070e+07,
          6.52315000e+05,   2.34260000e+04],
       [  4.90000000e+01,   3.00000000e+02,   6.68200000e+03,
          7.07000000e+02,   1.44000000e+02],
       [  0.00000000e+00,   1.80000000e+01,   7.35000000e+02,
          2.33000000e+02,   1.01000000e+02],
       [ -2.14000000e+02,  -4.82000000e+02,  -1.37000000e+03,
          2.59000000e+02,   1.97000000e+02],
       [  3.52000000e+02,   1.28600000e+03,   2.37390000e+04,
          3.13000000e+03,   4.15000000e+02],
       [  8.28000000e+04,   2.5951000

In [137]:
SUM_Bid3_13

array([[  0.00000000e+00,   7.13000000e+02,   1.89160000e+04,
          1.75400000e+03,   1.60000000e+01],
       [  0.00000000e+00,   7.50000000e+01,   1.44400000e+03,
          2.38000000e+02,   6.00000000e+00],
       [  0.00000000e+00,  -5.29000000e+02,  -5.04400000e+03,
          5.29000000e+02,   3.80000000e+01],
       [  0.00000000e+00,   1.17570000e+04,   7.59690000e+04,
          7.66100000e+03,   5.80000000e+01],
       [  9.00000000e+03,   5.47873000e+05,   1.56463660e+07,
          1.33333600e+06,   3.41900000e+03],
       [  3.00000000e+00,   1.54900000e+03,   1.83710000e+04,
          1.44500000e+03,   1.00000000e+01],
       [  0.00000000e+00,   9.80000000e+01,   1.91700000e+03,
          2.55000000e+02,   9.00000000e+00],
       [ -1.20000000e+01,  -3.36000000e+02,  -2.13200000e+03,
          9.52000000e+02,   3.00000000e+01],
       [  2.40000000e+01,   1.71120000e+04,   7.90210000e+04,
          6.70400000e+03,   3.20000000e+01],
       [  3.60000000e+03,   1.3474610

In [138]:
SUM_Bid1_16

array([[  0.00000000e+00,   7.13000000e+02,   1.89160000e+04,
          1.75400000e+03,   1.60000000e+01],
       [  0.00000000e+00,   3.87000000e+02,   7.34500000e+03,
          9.34000000e+02,   1.60000000e+01],
       [  0.00000000e+00,  -5.23000000e+02,  -2.12100000e+03,
          2.53000000e+02,   1.60000000e+01],
       [  0.00000000e+00,   5.75400000e+03,   3.42120000e+04,
          3.69300000e+03,   1.60000000e+01],
       [  0.00000000e+00,   3.24220000e+05,   1.22983390e+07,
          8.62108000e+05,   1.39500000e+03],
       [  3.00000000e+00,   1.54900000e+03,   1.83710000e+04,
          1.44500000e+03,   1.00000000e+01],
       [  0.00000000e+00,   6.82000000e+02,   7.53300000e+03,
          9.27000000e+02,   1.00000000e+01],
       [ -1.20000000e+01,  -2.75000000e+02,  -4.61100000e+03,
          4.31000000e+02,   1.10000000e+01],
       [  1.80000000e+01,   4.00700000e+03,   4.06460000e+04,
          3.52900000e+03,   1.10000000e+01],
       [  2.70000000e+03,   9.5761800

In [139]:
SUM_Bid2_16

array([[  0.00000000e+00,   7.13000000e+02,   1.89160000e+04,
          1.75400000e+03,   1.60000000e+01],
       [  0.00000000e+00,   1.71000000e+02,   2.55100000e+03,
          4.43000000e+02,   1.60000000e+01],
       [  0.00000000e+00,  -6.04000000e+02,  -5.61900000e+03,
          4.22000000e+02,   3.20000000e+01],
       [  0.00000000e+00,   9.05600000e+03,   5.59440000e+04,
          5.88800000e+03,   3.20000000e+01],
       [  0.00000000e+00,   5.05807000e+05,   1.49538110e+07,
          1.21662000e+06,   4.29100000e+03],
       [  3.00000000e+00,   1.54900000e+03,   1.83710000e+04,
          1.44500000e+03,   1.00000000e+01],
       [  0.00000000e+00,   3.69000000e+02,   3.27700000e+03,
          5.54000000e+02,   1.00000000e+01],
       [ -1.20000000e+01,  -6.10000000e+01,  -3.80900000e+03,
          7.84000000e+02,   2.10000000e+01],
       [  2.10000000e+01,   5.63900000e+03,   6.05720000e+04,
          5.25300000e+03,   2.10000000e+01],
       [  2.70000000e+03,   1.2148040

In [224]:
a = 0
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,120.000000,264.000000,1678.000000,309.000000,21.000000
1,0.175000,0.935606,0.474970,0.909385,1.000000
2,-1.583333,0.636364,-0.321216,0.624595,1.095238
3,4.391667,1.583333,3.323004,2.644013,1.333333
4,767.025000,279.280303,616.519070,353.773463,66.809524


In [225]:
a = 5
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,157.000000,324.000000,1047.000000,228.000000,19.000000
1,0.305732,0.848765,0.765043,0.793860,1.000000
2,-1.656051,0.231481,0.382999,0.407895,1.052632
3,4.356688,2.466049,2.942693,2.763158,1.473684
4,667.541401,286.543210,481.514804,444.337719,63.631579


In [226]:
a = 10
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,170.000000,418.000000,1180.000000,209.000000,13.000000
1,0.358824,0.813397,0.810169,0.665072,0.923077
2,-1.582353,0.246411,0.555085,-0.789474,0.923077
3,4.394118,2.846890,2.870339,3.569378,1.769231
4,654.470588,362.940191,467.951695,524.574163,73.076923


In [227]:
a = 15
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,125.000,379.000000,1094.000000,259.000000,5.0
1,0.408,0.812665,0.723949,0.335907,1.0
2,-1.448,0.200528,0.364717,-2.776062,1.0
3,4.048,2.720317,3.076782,5.069498,1.4
4,603.728,392.646438,535.381170,694.583012,76.6


In [228]:
a = 20
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,239.000000,494.000000,1900.000000,744.000000,4.00
1,0.577406,0.827935,0.742105,0.676075,1.00
2,-0.610879,0.216599,0.280000,-0.112903,1.00
3,3.531381,2.963563,3.306316,3.092742,1.75
4,504.598326,342.763158,567.086316,660.393817,78.25


In [229]:
a = 0
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,120.000000,264.000000,1678.000000,309.000000,21.000000
1,0.041667,0.875000,0.414779,0.423948,0.761905
2,-2.041667,1.420455,-0.013707,-0.129450,1.952381
3,5.858333,2.678030,4.531585,4.268608,2.571429
4,920.775000,384.503788,686.510727,716.501618,79.904762


In [230]:
a = 5
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,157.000000,324.000000,1047.000000,228.000000,19.000000
1,0.191083,0.712963,0.655205,0.521930,0.526316
2,-1.821656,0.688272,0.762178,-0.061404,1.578947
3,5.713376,3.725309,4.340974,4.916667,2.842105
4,837.394904,426.759259,593.951289,627.364035,99.894737


In [231]:
a = 10
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,170.000000,418.000000,1180.000000,209.000000,13.000000
1,0.229412,0.540670,0.726271,0.540670,0.384615
2,-1.623529,0.090909,1.165254,-0.473684,1.307692
3,5.582353,4.318182,4.105932,4.808612,3.076923
4,792.535294,563.349282,589.224576,632.507177,78.769231


In [232]:
a = 15
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,125.000,379.000000,1094.000000,259.000000,5.0
1,0.232,0.366755,0.672761,0.277992,0.4
2,-1.632,-0.472296,0.969835,-2.525097,1.4
3,5.320,5.087071,4.183729,6.100386,2.8
4,792.424,679.868074,634.206581,743.548263,91.6


In [233]:
a = 20
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,239.000000,494.000000,1900.000000,744.000000,4.0
1,0.330544,0.617409,0.598421,0.607527,0.0
2,-0.794979,-0.012146,0.838947,0.387097,1.0
3,4.828452,4.678138,4.415263,5.711022,3.5
4,756.251046,574.414980,673.321053,771.618280,0.0


In [234]:
a = 0
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,120.000000,264.000000,1678.000000,309.000000,21.000000
1,0.008333,0.325758,0.386770,0.271845,0.142857
2,-2.100000,0.659091,0.285459,0.110032,2.000000
3,6.858333,5.162879,5.618594,5.317152,3.571429
4,1027.625000,675.314394,724.151967,803.320388,47.047619


In [235]:
a = 5
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,157.000000,324.000000,1047.000000,228.000000,19.000000
1,0.076433,0.521605,0.513849,0.412281,0.421053
2,-2.000000,0.783951,0.912130,0.359649,2.000000
3,6.751592,5.070988,5.591213,5.916667,3.842105
4,911.987261,551.407407,683.702006,765.565789,188.368421


In [236]:
a = 10
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,170.000000,418.000000,1180.000000,209.000000,13.000000
1,0.105882,0.315789,0.595763,0.488038,0.153846
2,-1.823529,-0.131579,1.583898,0.014354,1.461538
3,6.623529,5.435407,5.250847,5.808612,4.076923
4,867.388235,706.193780,669.400847,719.401914,122.538462


In [237]:
a = 15
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,125.000,379.000000,1094.000000,259.000000,5.0
1,0.096,0.232190,0.489945,0.247104,0.2
2,-1.840,-0.535620,1.270567,-2.262548,1.6
3,6.344,6.187335,5.307130,7.100386,3.8
4,861.896,778.129288,721.987203,816.915058,159.0


In [238]:
a = 20
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,239.000000,494.000000,1900.000000,744.000000,4.0
1,0.083682,0.283401,0.364737,0.436828,0.0
2,-1.255230,-0.641700,0.772632,0.959677,1.0
3,5.853556,6.022267,5.706842,6.711022,4.5
4,849.380753,758.963563,795.253158,832.349462,0.0


In [194]:
a = 0
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,195.000000,674.000000,5406.000000,2262.000000,534.000000
1,0.564103,0.683976,0.795043,0.731211,0.876404
2,-0.784615,-0.048961,0.310211,-0.189655,0.672285
3,3.087179,2.698813,2.665742,3.328028,2.588015
4,749.061538,428.541543,402.312246,445.213086,157.878277


In [195]:
a = 5
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,184.000000,1231.000000,5766.000000,2221.000000,628.000000
1,0.538043,0.668562,0.626951,0.646556,0.902866
2,-0.918478,-0.046304,-0.494971,-0.572715,0.737261
3,3.635870,2.912266,3.459070,3.705538,2.982484
4,862.021739,461.032494,494.202914,495.978838,193.925159


In [196]:
a = 10
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,229.000000,1563.000000,6319.000000,2737.000000,742.000000
1,0.397380,0.621881,0.576990,0.510778,0.843666
2,-1.655022,-0.462572,-1.031651,-1.146145,0.590296
3,4.331878,3.275752,4.220763,4.488856,3.460916
4,1117.227074,499.202815,542.078019,605.262331,202.361186


In [197]:
a = 15
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,179.000000,1459.000000,5137.000000,1674.000000,426.000000
1,0.435754,0.569568,0.534943,0.487455,0.767606
2,-1.569832,-0.311857,-1.249562,-1.333333,0.241784
3,3.899441,3.270048,4.466225,4.666667,3.767606
4,1104.921788,548.881426,565.247421,626.163680,210.169014


In [198]:
a = 20
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,269.000000,2633.000000,8560.000000,1959.000000,413.000000
1,0.617100,0.555260,0.556659,0.482389,0.765133
2,-0.643123,-0.418914,-0.989136,-1.209801,0.491525
3,4.249071,3.283707,4.150818,4.533946,3.661017
4,897.750929,589.587543,554.322897,636.015314,215.711864


In [218]:
a = 0
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,195.000000,674.000000,5406.000000,2262.000000,534.000000
1,0.420513,0.459941,0.593600,0.593280,0.713483
2,-0.733333,-0.065282,0.495376,0.064987,1.037453
3,4.471795,4.176558,4.025342,4.690981,4.316479
4,1305.841026,613.406528,590.049760,662.931034,224.235955


In [217]:
a = 5
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,184.000000,1231.000000,5766.000000,2221.000000,628.000000
1,0.451087,0.488221,0.449532,0.377308,0.730892
2,-0.690217,0.117790,-0.511446,-0.932463,1.044586
3,4.858696,4.181966,4.847555,5.479514,4.687898
4,1521.239130,647.464663,644.247312,719.583521,258.154459


In [216]:
a = 10
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,229.000000,1563.000000,6319.000000,2737.000000,742.000000
1,0.275109,0.419706,0.383922,0.281695,0.588949
2,-1.458515,-0.512476,-1.138471,-1.455608,0.491914
3,5.406114,4.718490,5.631746,6.097917,5.252022
4,2001.270742,663.325016,697.837949,776.474973,221.088949


In [215]:
a = 15
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,179.000000,1459.000000,5137.000000,1674.000000,426.000000
1,0.206704,0.366690,0.355655,0.258662,0.464789
2,-1.776536,-0.245374,-1.386802,-1.741338,0.009390
3,5.486034,4.599726,5.937707,6.237754,5.502347
4,1975.821229,708.149417,725.454740,773.034050,197.368545


In [214]:
a = 20
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,269.000000,2633.000000,8560.000000,1959.000000,413.000000
1,0.516729,0.412457,0.388435,0.227667,0.527845
2,-0.074349,-0.251804,-1.009346,-1.594181,0.762712
3,5.260223,4.465629,5.558879,6.055641,4.915254
4,1310.226766,712.380175,713.174182,806.639102,189.559322


In [219]:
a = 0
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,195.000000,674.000000,5406.000000,2262.000000,534.000000
1,0.092308,0.393175,0.493156,0.254642,0.659176
2,-1.543590,0.227003,0.776915,0.009726,1.644195
3,6.348718,5.243323,5.169811,5.854996,5.387640
4,1691.497436,712.431751,689.929153,799.841733,303.256554


In [220]:
a = 5
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,184.000000,1231.000000,5766.000000,2221.000000,628.000000
1,0.320652,0.285946,0.339057,0.223323,0.652866
2,-0.375000,0.181966,-0.395942,-1.008555,1.609873
3,5.934783,5.268887,5.992542,6.749212,5.761146
4,1826.086957,800.441917,738.028963,825.590275,306.855096


In [221]:
a = 10
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,229.000000,1563.000000,6319.000000,2737.000000,742.000000
1,0.161572,0.252079,0.302105,0.118743,0.482480
2,-1.301310,-0.516955,-0.994619,-1.717209,0.886792
3,6.436681,5.838132,6.741257,7.399342,6.305930
4,2349.000000,793.773512,769.437727,863.396419,230.051213


In [222]:
a = 15
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,179.000000,1459.000000,5137.000000,1674.000000,426.000000
1,0.039106,0.224812,0.250925,0.129630,0.342723
2,-1.709497,-0.273475,-1.242943,-1.805257,0.206573
3,6.513966,5.713502,6.995523,7.439068,6.516432
4,2300.614525,813.540781,788.554993,874.297491,162.967136


In [223]:
a = 20
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,269.000000,2633.000000,8560.000000,1959.000000,413.000000
1,0.033457,0.214204,0.230724,0.080143,0.438257
2,-0.962825,-0.518420,-1.025234,-1.669219,1.096852
3,6.513011,5.667679,6.675350,7.161307,5.958838
4,1662.029740,829.913027,804.240654,884.928535,182.961259


In [179]:
a = 0
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,58.000000,202.000000,15159.000000,887.000000,105.000000
1,0.000000,0.405941,0.322647,0.527621,0.914286
2,-4.120690,-1.222772,0.386569,0.043968,0.885714
3,5.689655,2.975248,1.671878,2.413754,1.323810
4,1039.655172,627.584158,772.652682,568.461105,109.161905


In [180]:
a = 5
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,49.000000,300.000000,6682.000000,707.000000,144.000000
1,0.000000,0.306667,0.285394,0.536068,0.840278
2,-4.367347,-1.226667,-0.141874,0.292786,0.763889
3,6.183673,3.000000,2.418438,3.256011,1.708333
4,1322.448980,691.893333,739.685124,622.459689,185.868056


In [181]:
a = 10
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,33.000000,296.000000,6940.000000,523.000000,110.000000
1,0.000000,0.510135,0.371614,0.491396,0.909091
2,-4.818182,-0.847973,0.067147,0.133843,1.045455
3,6.090909,3.141892,2.261527,2.934990,1.790909
4,1172.727273,576.006757,724.554611,611.967495,211.945455


In [182]:
a = 15
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,19.000000,318.000000,6330.000000,409.000000,20.00
1,0.000000,0.509434,0.304107,0.466993,0.95
2,-5.157895,-0.959119,-0.061611,0.586797,0.85
3,6.315789,3.374214,2.056714,2.190709,1.70
4,947.368421,553.761006,754.659400,641.166259,240.05


In [183]:
a = 20
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,17.000000,1523.000000,23766.000000,184.000000,19.000000
1,0.117647,0.205515,0.092569,0.673913,1.000000
2,-5.294118,-0.454366,-0.350837,-0.027174,1.000000
3,6.647059,2.670387,2.139737,3.059783,1.684211
4,801.058824,753.053841,852.015274,537.407609,379.263158


In [184]:
a = 0
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,58.000000,202.000000,15159.000000,887.000000,105.000000
1,0.000000,0.074257,0.068804,0.280722,0.761905
2,-4.120690,-1.792079,0.404842,0.111612,1.476190
3,6.689655,4.252475,2.716274,3.550169,2.380952
4,1287.931034,837.915842,870.367900,735.417136,223.104762


In [185]:
a = 5
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,49.000000,300.000000,6682.000000,707.000000,144.000000
1,0.000000,0.060000,0.109997,0.329562,0.701389
2,-4.367347,-1.606667,-0.205028,0.366337,1.368056
3,7.183673,4.286667,3.552679,4.427157,2.881944
4,1689.795918,865.033333,852.009129,750.155587,313.937500


In [186]:
a = 10
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,33.000000,296.000000,6940.000000,523.000000,110.000000
1,0.000000,0.101351,0.173199,0.390057,0.745455
2,-4.818182,-1.371622,0.138617,0.495220,1.645455
3,7.090909,4.601351,3.400720,4.080306,2.863636
4,1663.636364,830.351351,827.614553,695.361377,385.218182


In [187]:
a = 15
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,19.000000,318.000000,6330.000000,409.000000,20.00
1,0.000000,0.056604,0.136177,0.278729,0.70
2,-5.157895,-1.864780,-0.004265,0.814181,1.35
3,7.315789,4.905660,3.122433,3.339853,2.95
4,1184.210526,857.767296,859.090521,732.154034,321.30


In [188]:
a = 20
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,17.000000,1523.000000,23766.000000,184.000000,19.000000
1,0.000000,0.038083,0.033704,0.445652,0.578947
2,-5.647059,-0.559422,-0.344357,0.375000,0.842105
3,7.882353,3.759028,3.163006,4.152174,3.000000
4,1323.529412,870.704531,883.326811,635.086957,369.421053


In [189]:
a = 0
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,0.000000,713.000000,18916.000000,1754.000000,16.0000
1,NaN,0.105189,0.076337,0.135690,0.3750
2,NaN,-0.741935,-0.266653,0.301596,2.3750
3,NaN,16.489481,4.016124,4.367731,3.6250
4,inf,768.405330,827.149820,760.168757,213.6875


In [190]:
a = 5
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,3.0,1549.000000,18371.000000,1445.000000,10.0
1,0.0,0.063267,0.104349,0.176471,0.9
2,-4.0,-0.216914,-0.116052,0.658824,3.0
3,8.0,11.047127,4.301399,4.639446,3.2
4,1200.0,869.890897,835.485766,814.588927,395.8


In [191]:
a = 10
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,5.0,2197.000000,16557.000000,1724.000000,9.000000
1,0.0,0.112426,0.099052,0.132251,1.000000
2,-4.0,-0.373691,-0.272815,0.428074,3.000000
3,8.0,8.776513,4.506312,4.929234,3.333333
4,900.0,849.998179,837.544603,838.254640,307.000000


In [192]:
a = 15
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,1.0,1455.000000,9482.000000,719.000000,9.000000
1,0.0,0.114777,0.098397,0.201669,1.000000
2,-4.0,-0.518213,0.024256,0.915160,3.111111
3,8.0,7.281100,4.564965,4.897079,3.777778
4,900.0,859.645361,859.832947,821.445063,345.666667


In [193]:
a = 20
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,7.000000,2126.000000,19256.000000,855.000000,21.000000
1,0.000000,0.079022,0.045752,0.223392,1.000000
2,-1.285714,-1.125118,-0.610251,1.072515,3.000000
3,5.000000,12.159925,4.354331,4.612865,4.000000
4,900.000000,855.440734,833.490237,775.597661,361.619048


In [199]:
a = 0
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,0.0,713.000000,18916.000000,1754.000000,16.0000
1,NaN,0.542777,0.388296,0.532497,1.0000
2,NaN,-0.733520,-0.112127,0.144242,1.0000
3,NaN,8.070126,1.808628,2.105473,1.0000
4,NaN,454.726508,650.155371,491.509692,87.1875


In [200]:
a = 5
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,3.0,1549.000000,18371.000000,1445.000000,10.0
1,0.0,0.440284,0.410048,0.641522,1.0
2,-4.0,-0.177534,-0.250993,0.298270,1.1
3,6.0,2.586830,2.212509,2.442215,1.1
4,900.0,618.216914,662.002341,511.785467,99.4


In [201]:
a = 10
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,5.0,2197.000000,16557.000000,1724.000000,9.000000
1,0.0,0.352754,0.346440,0.571346,1.000000
2,-4.0,-0.442421,-0.405569,0.317285,1.000000
3,6.0,2.672736,2.411246,2.603828,1.333333
4,900.0,706.524351,693.269131,581.899072,101.444444


In [202]:
a = 15
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,1.0,1455.000000,9482.000000,719.000000,9.000000
1,0.0,0.345704,0.402447,0.627260,1.000000
2,-4.0,-0.646048,-0.220839,0.336579,1.111111
3,6.0,2.827491,2.500000,2.606398,1.777778
4,900.0,704.427491,680.351719,527.203060,190.000000


In [203]:
a = 20
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,7.000000,2126.000000,19256.000000,855.000000,21.000000
1,0.000000,0.349012,0.267086,0.707602,1.000000
2,-1.285714,-0.553622,-0.353396,0.412865,1.000000
3,3.000000,5.810913,2.325197,2.369591,2.000000
4,900.000000,729.951082,740.342491,472.694737,238.619048


In [204]:
a = 0
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,0.0,713.000000,18916.000000,1754.000000,16.0000
1,NaN,0.239832,0.134859,0.252566,1.0000
2,NaN,-0.847125,-0.297050,0.240593,2.0000
3,NaN,12.701262,2.957496,3.356899,2.0000
4,NaN,709.406732,790.537693,693.625998,268.1875


In [205]:
a = 5
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,3.0,1549.000000,18371.000000,1445.000000,10.0
1,0.0,0.238218,0.178379,0.383391,1.0
2,-4.0,-0.039380,-0.207338,0.542561,2.1
3,7.0,3.640413,3.297153,3.635294,2.1
4,900.0,784.250484,796.188721,705.230450,344.2


In [206]:
a = 10
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,5.0,2197.000000,16557.000000,1724.000000,9.000000
1,0.0,0.222121,0.160838,0.268561,1.000000
2,-4.0,-0.275376,-0.349037,0.291183,2.000000
3,7.0,3.679563,3.490004,3.916473,2.333333
4,900.0,799.650432,799.606330,774.359049,240.777778


In [207]:
a = 15
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,1.0,1455.000000,9482.000000,719.000000,9.000000
1,0.0,0.189003,0.181080,0.385257,1.000000
2,-4.0,-0.535395,-0.076144,0.734353,2.111111
3,7.0,3.837801,3.562012,3.855355,2.777778
4,900.0,817.999313,822.825670,714.490960,287.222222


In [208]:
a = 20
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,7.000000,2126.000000,19256.000000,855.000000,21.000000
1,0.000000,0.235183,0.125415,0.402339,1.000000
2,-1.285714,-0.459548,-0.455287,0.839766,2.000000
3,4.000000,9.246472,3.349813,3.592982,3.000000
4,900.000000,799.073377,810.771656,691.858480,305.619048


In [169]:
a = 0
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,0.000000,713.000000,18916.000000,1754.000000,16.0000
1,NaN,0.105189,0.076337,0.135690,0.3750
2,NaN,-0.741935,-0.266653,0.301596,2.3750
3,NaN,16.489481,4.016124,4.367731,3.6250
4,inf,768.405330,827.149820,760.168757,213.6875


In [168]:
a = 5
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,3.0,1549.000000,18371.000000,1445.000000,10.0
1,0.0,0.063267,0.104349,0.176471,0.9
2,-4.0,-0.216914,-0.116052,0.658824,3.0
3,8.0,11.047127,4.301399,4.639446,3.2
4,1200.0,869.890897,835.485766,814.588927,395.8


In [167]:
a = 10
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,5.0,2197.000000,16557.000000,1724.000000,9.000000
1,0.0,0.112426,0.099052,0.132251,1.000000
2,-4.0,-0.373691,-0.272815,0.428074,3.000000
3,8.0,8.776513,4.506312,4.929234,3.333333
4,900.0,849.998179,837.544603,838.254640,307.000000


In [166]:
a = 15
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,1.0,1455.000000,9482.000000,719.000000,9.000000
1,0.0,0.114777,0.098397,0.201669,1.000000
2,-4.0,-0.518213,0.024256,0.915160,3.111111
3,8.0,7.281100,4.564965,4.897079,3.777778
4,900.0,859.645361,859.832947,821.445063,345.666667


In [165]:
a = 20
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,7.000000,2126.000000,19256.000000,855.000000,21.000000
1,0.000000,0.079022,0.045752,0.223392,1.000000
2,-1.285714,-1.125118,-0.610251,1.072515,3.000000
3,5.000000,12.159925,4.354331,4.612865,4.000000
4,900.000000,855.440734,833.490237,775.597661,361.619048
